# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [56]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys_use import g_key


### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [57]:
cities_vac=pd.read_csv("./output_data/cities.csv")
cities_vac.head()



,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,Punta Arenas,-53.1500,-70.9167,7.06,57,75,6.69,CL,1624128629
1,1,Busselton,-33.6500,115.3333,13.38,82,100,4.89,AU,1624128398
2,2,Qasigiannguit,68.8193,-51.1922,4.22,65,100,1.57,GL,1624128629
3,3,Tasiilaq,65.6145,-37.6368,3.13,68,100,2.00,GL,1624128629
4,4,Ushuaia,-54.8000,-68.3000,5.81,75,75,8.23,AR,1624128630


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [58]:
# Configure gmaps
gmaps.configure(api_key=g_key)

# Store latitude and longitude in locations
locations=cities_vac[["Lat","Lng"]]
locations.head()

,Lat,Lng
0,-53.1500,-70.9167
1,-33.6500,115.3333
2,68.8193,-51.1922
3,65.6145,-37.6368
4,-54.8000,-68.3000


In [59]:
weight = cities_vac["Humidity"]
weight.head()

0    57
1    82
2    65
3    68
4    75
Name: Humidity, dtype: int64

In [60]:
# Customize the size of the figure
figure_layout = {
    'width': '600px',
    'height': '600px',
    'border': '1px solid black',
    'padding': '1px',
    'margin': '0 auto 0 auto'
}
fig = gmaps.figure(layout=figure_layout)
heatmap = gmaps.heatmap_layer(locations,weights=weight)
heatmap.max_intensity = cities_vac["Humidity"].max()

fig.add_layer(heatmap)
fig

Figure(layout=FigureLayout(border='1px solid black', height='600px', margin='0 auto 0 auto', padding='1px', wi…

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [61]:
#filtering out vacation criteria
hotel_df= cities_vac.loc[(cities_vac['Humidity'] < 60) & (cities_vac['Humidity'] > 40) & (cities_vac['Max Temp'] < 40) & (cities_vac['Max Temp'] > 25) &(cities_vac['Cloudiness'] < 50) ]
hotel_df
#no null values, dont need to drop

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
165,165,Arkadelphia,34.1209,-93.0538,33.82,58,1,0.00,US,1624128667
182,182,San Patricio,28.0170,-97.5169,33.93,45,24,4.75,US,1624128671
235,235,San Cristobal,7.7669,-72.2250,28.10,50,33,3.20,VE,1624128685
318,318,Nouakchott,18.0858,-15.9785,28.96,58,0,9.75,MR,1624128704
325,325,Umluj,25.0213,37.2685,30.16,45,0,3.81,SA,1624128706
330,330,Rome,43.2128,-75.4557,29.35,52,20,0.45,US,1624128707
334,334,Lincoln,40.8000,-96.6670,31.74,45,1,2.68,US,1624128676
392,392,Dzilam González,21.2833,-88.9333,34.51,41,37,2.86,MX,1624128722
394,394,Oodweyne,9.4092,45.0640,25.16,46,21,9.93,SO,1624128723
419,419,Conceição do Araguaia,-8.2578,-49.2647,34.07,43,0,4.12,BR,1624128483


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [62]:
hotel_df["Hotel Name"]=""
hotel_df["Country"]=""
hotel_df

<ipython-input-62-0ae572e6e363>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hotel_df["Hotel Name"]=""
<ipython-input-62-0ae572e6e363>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hotel_df["Country"]=""


,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
165,165,Arkadelphia,34.1209,-93.0538,33.82,58,1,0.00,,1624128667,
182,182,San Patricio,28.0170,-97.5169,33.93,45,24,4.75,,1624128671,
235,235,San Cristobal,7.7669,-72.2250,28.10,50,33,3.20,,1624128685,
318,318,Nouakchott,18.0858,-15.9785,28.96,58,0,9.75,,1624128704,
325,325,Umluj,25.0213,37.2685,30.16,45,0,3.81,,1624128706,
330,330,Rome,43.2128,-75.4557,29.35,52,20,0.45,,1624128707,
334,334,Lincoln,40.8000,-96.6670,31.74,45,1,2.68,,1624128676,
392,392,Dzilam González,21.2833,-88.9333,34.51,41,37,2.86,,1624128722,
394,394,Oodweyne,9.4092,45.0640,25.16,46,21,9.93,,1624128723,
419,419,Conceição do Araguaia,-8.2578,-49.2647,34.07,43,0,4.12,,1624128483,


In [63]:
# find the closest hotel of each type to coordinates

base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
params = {
    "radius": 5000,
    "type": "hotel",
    "key": g_key}

# use iterrows to iterate through pandas dataframe
for index, row in hotel_df.iterrows():

    # get location from df
    latitude = row['Lat']
    longitude = row["Lng"]
    location= f"{latitude},{longitude}"


    # add location to params dict
    params['location'] = location

    # assemble url and make API request
    print(f"Retrieving Results for Index {index}: {row[1]}.")
    response = requests.get(base_url, params=params).json()
    
    # extract results
    results = response['results']
    
    try:
        print(f"Closest Hotel is {results[1]['name']}.")
        hotel_df.loc[index, 'Hotel Name'] = results[1]['name']
        hotel_df.loc[index, 'Country'] = results[1]["plus_code"]["compound_code"]
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")
        
    print("------------")

Retrieving Results for Index 165: Arkadelphia.
Closest Hotel is KDEL-FM Arkadelphia.
------------
Retrieving Results for Index 182: San Patricio.


C:\Users\insiy\anaconda3\lib\site-packages\pandas\core\indexing.py:1765: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(loc, value)


Closest Hotel is Sinton High School.
------------
Retrieving Results for Index 235: San Cristobal.
Closest Hotel is Banco Provincial.
------------
Retrieving Results for Index 318: Nouakchott.
Closest Hotel is Marche Capitale.
------------
Retrieving Results for Index 325: Umluj.
Closest Hotel is Moon light Furnished Units.
Missing field/result... skipping.
------------
Retrieving Results for Index 330: Rome.
Closest Hotel is Wingate By Wyndham Rome.
------------
Retrieving Results for Index 334: Lincoln.
Closest Hotel is Courtyard by Marriott Lincoln Downtown/Haymarket.
------------
Retrieving Results for Index 392: Dzilam González.
Closest Hotel is Presidencia Municipal Dzilam González.
------------
Retrieving Results for Index 394: Oodweyne.
Closest Hotel is Xasaasi Media.
------------
Retrieving Results for Index 419: Conceição do Araguaia.
Closest Hotel is Bradesco Dia&Noite.
------------
Retrieving Results for Index 430: Longview.
Closest Hotel is Homewood Suites by Hilton Longvi

In [64]:
hotel_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
165,165,Arkadelphia,34.1209,-93.0538,33.82,58,1,0.00,"4W6X+9R Arkadelphia, AR, USA",1624128667,KDEL-FM Arkadelphia
182,182,San Patricio,28.0170,-97.5169,33.93,45,24,4.75,"2GR3+65 Sinton, TX, USA",1624128671,Sinton High School
235,235,San Cristobal,7.7669,-72.2250,28.10,50,33,3.20,"QQ8P+JG San Cristobal, Táchira, Venezuela",1624128685,Banco Provincial
318,318,Nouakchott,18.0858,-15.9785,28.96,58,0,9.75,"32PC+F4 Nouakchott, Mauritania",1624128704,Marche Capitale
325,325,Umluj,25.0213,37.2685,30.16,45,0,3.81,,1624128706,Moon light Furnished Units
330,330,Rome,43.2128,-75.4557,29.35,52,20,0.45,"6H5P+PR Rome, NY, USA",1624128707,Wingate By Wyndham Rome
334,334,Lincoln,40.8000,-96.6670,31.74,45,1,2.68,"R78R+W4 Lincoln, NE, USA",1624128676,Courtyard by Marriott Lincoln Downtown/Haymarket
392,392,Dzilam González,21.2833,-88.9333,34.51,41,37,2.86,"73JC+23 Dzilam González, Yuc., Mexico",1624128722,Presidencia Municipal Dzilam González
394,394,Oodweyne,9.4092,45.0640,25.16,46,21,9.93,"C357+WQ Oodweyne, Somalia",1624128723,Xasaasi Media
419,419,Conceição do Araguaia,-8.2578,-49.2647,34.07,43,0,4.12,"PPPP+Q7 Centro, Conceição do Araguaia - PA, Br...",1624128483,Bradesco Dia&Noite


In [68]:
hotel_df_copy=hotel_df

In [69]:
#split country columns
hotel_df_copy[['Location','State','Country']] = hotel_df.Country.str.split(",",expand=True)
hotel_df_copy

C:\Users\insiy\anaconda3\lib\site-packages\pandas\core\frame.py:3065: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[k1] = value[k2]


,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name,Location,State
165,165,Arkadelphia,34.1209,-93.0538,33.82,58,1,0.00,USA,1624128667,KDEL-FM Arkadelphia,4W6X+9R Arkadelphia,AR
182,182,San Patricio,28.0170,-97.5169,33.93,45,24,4.75,USA,1624128671,Sinton High School,2GR3+65 Sinton,TX
235,235,San Cristobal,7.7669,-72.2250,28.10,50,33,3.20,Venezuela,1624128685,Banco Provincial,QQ8P+JG San Cristobal,Táchira
318,318,Nouakchott,18.0858,-15.9785,28.96,58,0,9.75,None,1624128704,Marche Capitale,32PC+F4 Nouakchott,Mauritania
325,325,Umluj,25.0213,37.2685,30.16,45,0,3.81,None,1624128706,Moon light Furnished Units,,None
330,330,Rome,43.2128,-75.4557,29.35,52,20,0.45,USA,1624128707,Wingate By Wyndham Rome,6H5P+PR Rome,NY
334,334,Lincoln,40.8000,-96.6670,31.74,45,1,2.68,USA,1624128676,Courtyard by Marriott Lincoln Downtown/Haymarket,R78R+W4 Lincoln,NE
392,392,Dzilam González,21.2833,-88.9333,34.51,41,37,2.86,Mexico,1624128722,Presidencia Municipal Dzilam González,73JC+23 Dzilam González,Yuc.
394,394,Oodweyne,9.4092,45.0640,25.16,46,21,9.93,None,1624128723,Xasaasi Media,C357+WQ Oodweyne,Somalia
419,419,Conceição do Araguaia,-8.2578,-49.2647,34.07,43,0,4.12,Brazil,1624128483,Bradesco Dia&Noite,PPPP+Q7 Centro,Conceição do Araguaia - PA


In [70]:
#drop empty rows
hotel_df_copy=hotel_df_copy.dropna()
hotel_df_copy

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name,Location,State
165,165,Arkadelphia,34.1209,-93.0538,33.82,58,1,0.00,USA,1624128667,KDEL-FM Arkadelphia,4W6X+9R Arkadelphia,AR
182,182,San Patricio,28.0170,-97.5169,33.93,45,24,4.75,USA,1624128671,Sinton High School,2GR3+65 Sinton,TX
235,235,San Cristobal,7.7669,-72.2250,28.10,50,33,3.20,Venezuela,1624128685,Banco Provincial,QQ8P+JG San Cristobal,Táchira
330,330,Rome,43.2128,-75.4557,29.35,52,20,0.45,USA,1624128707,Wingate By Wyndham Rome,6H5P+PR Rome,NY
334,334,Lincoln,40.8000,-96.6670,31.74,45,1,2.68,USA,1624128676,Courtyard by Marriott Lincoln Downtown/Haymarket,R78R+W4 Lincoln,NE
392,392,Dzilam González,21.2833,-88.9333,34.51,41,37,2.86,Mexico,1624128722,Presidencia Municipal Dzilam González,73JC+23 Dzilam González,Yuc.
419,419,Conceição do Araguaia,-8.2578,-49.2647,34.07,43,0,4.12,Brazil,1624128483,Bradesco Dia&Noite,PPPP+Q7 Centro,Conceição do Araguaia - PA
430,430,Longview,32.5007,-94.7405,34.19,51,20,0.89,USA,1624128731,Homewood Suites by Hilton Longview,G62X+GH Longview,TX
451,451,Broome,42.2506,-75.8330,28.94,54,1,0.45,USA,1624128736,Chenango Valley State Park,6588+68 Chenango Forks,NY
571,571,Topolobampo,25.6000,-109.0500,34.47,49,3,6.06,Mexico,1624128765,Hotel Marina Topolobampo,HWXX+X3 Topolobampo,Sinaloa


In [71]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
Name: {Hotel Name}
City: {City}
Country: {Country}
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df_copy.iterrows()]
locations = hotel_df_copy[["Lat", "Lng"]]


In [72]:
# Add marker layer ontop of heat map
marker_layer=gmaps.marker_layer(locations,hover_text=hotel_info)


fig.add_layer(marker_layer)
fig

Figure(layout=FigureLayout(border='1px solid black', height='600px', margin='0 auto 0 auto', padding='1px', wi…